In [12]:
from datasets.dataset import Dataset

import os

images_paths = []


for dataset in Dataset:
    partial_images_paths = []
    dataset_path = "./datasets/{}".format(dataset.name)
    for path in os.listdir(dataset_path):
        images = os.listdir(os.path.join(dataset_path, path))
        partial_images_paths += [os.path.join(dataset_path, path, image) for image in images]
    print(f"{dataset.name}: {len(partial_images_paths)} images")
    images_paths += partial_images_paths
    
print('---------------------------------')
print(f"Total {len(images_paths)} images")
    

KITTI: 2011 images
LASOT: 2000 images
MOT20: 2000 images
---------------------------------
Total 6011 images


In [3]:
len(images_paths)

14894